In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
t0 = time.time()
from matplotlib import pyplot as plt
import os
import re
import scipy.signal
from kaveh.toolbox import common_avg_ref, butter_bandpass_filter, find_files_regex
import OpenEphys

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
num_electrode_contacts = 7 # 7 for heptodes; 4 for tetrodes
source_path = '/path/to/raw/open ephys/directory'


# Load the input data
Load the raw channel voltage here. Save it in `voltage_trace` as a 1d numpy array. Save sampling frequency in the variable `Fs`

In [ ]:
f_names = []
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        f_regex = re.compile(r".*CH(\d|\d\d)\.continuous$")
        if f_regex.match(filename):
            f_names = f_names + [os.path.join(root, filename)]
f_names = np.array(f_names)
chans = [int(f.split('.')[-2].split('_')[-1][2:]) for f in f_names]
f_names = f_names[np.argsort(chans)] # now sorted by channel number

chan_data = []
for contact in range(num_electrode_contacts):
    contact_reps = []
    for i in range(contact*4, contact*4 + 4):
        print('Reading {}...'.format(f_names[i]))
        file_content = OpenEphys.load(f_names[i])
        Fs = float(file_content['header']['sampleRate'])
        signal_filtered = butter_bandpass_filter(file_content['data'], 200, 10000, Fs, order=2 )
        contact_reps.append(signal_filtered)
    contact_reps = np.array(contact_reps)
    chan_data.append(np.mean(contact_reps, axis=0))
    print('-----------------------------------------------')
chan_data = np.array(chan_data)
contact_reps = None

common_avg_ref(chan_data)
chan_data = scipy.signal.detrend(chan_data)

### Saving:

In [ ]:
chan_filtered_fname = os.path.join(source_path, 'channels_filtered.npy')
print('Saving filtered data to {}'.format(chan_filtered_fname))
np.save(chan_filtered_fname, chan_data)

In [ ]:
chan_index = 0 # index of contact (zero indexed; e.g. for a tetrode it should be 0, 1, 2, or 3)
voltage_trace = np.squeeze(chan_data[chan_index, :])